In [1]:
import torch
from torchvision import datasets, transforms

# Veri dönüşümleri
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Veri seti için normalize işlemi
])

# SVHN veri setini yükleme
train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=transform)

# Veri yükleyicilerini oluşturma
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


100%|██████████| 182040794/182040794 [00:16<00:00, 11118597.56it/s]


100%|██████████| 64275384/64275384 [00:01<00:00, 43776239.71it/s]


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Sinir ağı modeli
class ClassifierModule(nn.Module):
    def __init__(self):
        super(ClassifierModule, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)  # Giriş boyutunu güncelle
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ClassifierModule().to(device)

# Loss fonksiyonu ve optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Eğitim döngüsü
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(dataloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

# Evaluation fonksiyonu
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(dataloader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            running_loss += loss.item()
            _, predicted = output.max(1)
            correct += predicted.eq(target).sum().item()
            total += target.size(0)
    return running_loss / len(dataloader), correct / total

# Veri yükleyicilerini oluşturma
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model eğitimi
num_epochs = 100

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Epoch 1/100: Train Loss: 2.0656, Test Loss: 2.0047, Test Accuracy: 0.4532
Epoch 2/100: Train Loss: 2.0008, Test Loss: 1.9828, Test Accuracy: 0.4761
Epoch 3/100: Train Loss: 1.9892, Test Loss: 2.0123, Test Accuracy: 0.4464
Epoch 4/100: Train Loss: 1.9946, Test Loss: 1.9838, Test Accuracy: 0.4763
Epoch 5/100: Train Loss: 1.9922, Test Loss: 1.9917, Test Accuracy: 0.4683
Epoch 6/100: Train Loss: 1.9891, Test Loss: 1.9970, Test Accuracy: 0.4635
Epoch 7/100: Train Loss: 1.9902, Test Loss: 1.9729, Test Accuracy: 0.4875
Epoch 8/100: Train Loss: 1.9966, Test Loss: 1.9761, Test Accuracy: 0.4844
Epoch 9/100: Train Loss: 2.0091, Test Loss: 2.0357, Test Accuracy: 0.4252
Epoch 10/100: Train Loss: 2.0076, Test Loss: 1.9803, Test Accuracy: 0.4806
Epoch 11/100: Train Loss: 2.0014, Test Loss: 1.9996, Test Accuracy: 0.4613
Epoch 12/100: Train Loss: 2.0000, Test Loss: 2.0218, Test Accuracy: 0.4390
Epoch 13/100: Train Loss: 2.0130, Test Loss: 2.0226, Test Accuracy: 0.4382
Epoch 14/100: Train Loss: 2.0145, 

In [3]:
def apply_lorentz_transformation(data):
    # Lorentz dönüşümü uygulama kodu
    beta = 0.99
    gamma = 1 / (torch.sqrt(torch.tensor(1 - beta**2)) if beta < 1 else float('inf'))  # Lorentz faktörü

    data_tensor = torch.tensor(data, dtype=torch.float32)  # Data'yı Tensor'a dönüştür
    transformed_data = gamma * (data_tensor - beta * data_tensor)
    return transformed_data

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


# Eğitim döngüsü
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(dataloader):
        data, target = data.to(device), target.to(device)

        # Lorentz dönüşümü uygulama
        data = apply_lorentz_transformation(data)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

# Evaluation fonksiyonu
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(dataloader):
            data, target = data.to(device), target.to(device)

            # Lorentz dönüşümü uygulama
            data = apply_lorentz_transformation(data)

            output = model(data)
            loss = criterion(output, target)
            running_loss += loss.item()
            _, predicted = output.max(1)
            correct += predicted.eq(target).sum().item()
            total += target.size(0)
    return running_loss / len(dataloader), correct / total

# Model eğitimi
num_epochs = 100

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


<ipython-input-3-ca63d6f88a5e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_tensor = torch.tensor(data, dtype=torch.float32)  # Data'yı Tensor'a dönüştür


Epoch 1/100: Train Loss: 1.9604, Test Loss: 1.9619, Test Accuracy: 0.4990
Epoch 2/100: Train Loss: 1.9537, Test Loss: 1.9746, Test Accuracy: 0.4861
Epoch 3/100: Train Loss: 1.9557, Test Loss: 1.9580, Test Accuracy: 0.5029
Epoch 4/100: Train Loss: 1.9563, Test Loss: 1.9606, Test Accuracy: 0.5003
Epoch 5/100: Train Loss: 1.9567, Test Loss: 1.9577, Test Accuracy: 0.5034
Epoch 6/100: Train Loss: 1.9533, Test Loss: 1.9600, Test Accuracy: 0.5010
Epoch 7/100: Train Loss: 1.9508, Test Loss: 1.9527, Test Accuracy: 0.5083
Epoch 8/100: Train Loss: 1.9546, Test Loss: 1.9530, Test Accuracy: 0.5079
Epoch 9/100: Train Loss: 1.9520, Test Loss: 1.9681, Test Accuracy: 0.4928
Epoch 10/100: Train Loss: 1.9507, Test Loss: 1.9740, Test Accuracy: 0.4869
Epoch 11/100: Train Loss: 1.9509, Test Loss: 1.9568, Test Accuracy: 0.5041
Epoch 12/100: Train Loss: 1.9531, Test Loss: 1.9632, Test Accuracy: 0.4978
Epoch 13/100: Train Loss: 1.9564, Test Loss: 1.9570, Test Accuracy: 0.5039
Epoch 14/100: Train Loss: 1.9580, 